### The notebook tests the runtime if running the emulator within the numpy, the C++ standalone or the C++ standalone backend with multithreading enabled.

# Single Source Shortest Path


Below is an implementation of a Wavefront algorithm using spiking neurons. \
An in depth description of such an algorithm can be viewed in [0].\
The algorithm is implemented using the [Loihi emulator](https://github.com/sagacitysite/brian2_loihi) described in [1].\
Concrete values and the learning rule are taken from the PathPlanning library provided by Intel via their NxSDK Apps package (you have to be INRC member to get the package).

In summary the idea is that a wave of neuron spikes is running trough a lattice graph - a lattice graph is a rectangle of nodes where every node is, if possible, connected to their four surounding neigbours - until the spike front hits the target node / neuron. \
Path backtracing is done via the synaptic weights, which are altered by an anti hebbian leraning rule to enable a change in the weight that can be backtraced to reconstruct the shortest path.

[0] Ponulak F, Hopfield JJ. Rapid, parallel path planning by propagating wavefronts of spiking neural activity. Front Comput Neurosci. 2013 Jul 18;7:98. doi: 10.3389/fncom.2013.00098. PMID: 23882213; PMCID: PMC3714542.

[1] Brian2Loihi: An emulator for the neuromorphic chip Loihi using the spiking neural network simulator Brian
Carlo Michaelis, Andrew B. Lehr, Winfried Oed, Christian Tetzlaff





# Simulation Parameters

In [1]:
# ------------------------------
#       import libs
# ------------------------------
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import time as time


# ------------------------------
#       variables
# ------------------------------
# Graph size
N = 300

# where is the C++ code stored in standalone mode?
brian_code_dir = 'Cpp_target_code' 

# the maximal length for a path in a lattice graph is N*2,
# which would be a path going from one corner to the one in front of.
# Therefore the simulation needs to be a max as long as N*2 steps
# since then the spikefront would have hit the target definitely.
net_runtime = N*2 

# target and source neuron
# it is the number of the neuron inside the graph (counting starts at zero!).
# To generate one of the longest possible paths do:
source_neuron = N*N-1
target_neuron = 0

# Lattice graph making functions

In [2]:
def fully_connected_nxn_graph(N):
    """
    Creates a fully connected N x N Graph connection matrix that uses an insane amount of memory !
    """
    # make empty matrix
    A = np.zeros((N*N,N*N), dtype=int)
    
    # make connections
    for n in range(N*N):
        # left neigbour
        if n%N > 0:
            A[n][n-1] = 1
        
        # right neigbour
        if ((n+1)/N) - int((n+1)/N) > 0:
            A[n][n+1] = 1
        
        # upper neigbour
        if n >= N:
            A[n][n-N] = 1
        
        # lower neigbour
        if n < N*N - N:
            A[n][n+N] = 1
        
    return A



def fully_connected_nxn_graph_it(N):
    """
    Creates a fully connected N x N Graph represented as indices and targets
    """
    # make lists
    indices = []
    targets = []
    
    # make connections
    for n in range(N*N):
        # left neigbour
        if n%N > 0:
            #A[n][n-1] = 1
            indices.append(n)
            targets.append(n-1)
        
        # right neigbour
        if ((n+1)/N) - int((n+1)/N) > 0:
            #A[n][n+1] = 1
            indices.append(n)
            targets.append(n+1)
        
        # upper neigbour
        if n >= N:
            #A[n][n-N] = 1
            indices.append(n)
            targets.append(n-N)
            
        
        # lower neigbour
        if n < N*N - N:
            #A[n][n+N] = 1
            indices.append(n)
            targets.append(n+N)
        
    return indices, targets

# Define a Graph and measure memory usage

In [3]:
# make it for adjacancy matrix
#C = fully_connected_nxn_graph(N)

#size_in_bytes = C.nbytes
#size_in_Kbytes = size_in_bytes/1024
#size_in_Mbytes = size_in_Kbytes/1024
#size_in_Gbytes = size_in_Mbytes/1024

#print('Graph memory size adjacancy matrix')
#print('byte: ', size_in_bytes)
#print('Kbytes : ', size_in_Kbytes)
#print('Mbytes : ', size_in_Mbytes)
#print('Gbytes : ', size_in_Gbytes)
#print()
#print()
#print()


# make it for indices targets versi0n
indices, targets = fully_connected_nxn_graph_it(N)

size_in_bytes = np.array(indices).nbytes
size_in_Kbytes = size_in_bytes/1024
size_in_Mbytes = size_in_Kbytes/1024
size_in_Gbytes = size_in_Mbytes/1024

print('Graph memory size indices, targets')
print('byte: ', size_in_bytes)
print('Kbytes : ', size_in_Kbytes)
print('Mbytes : ', size_in_Mbytes)
print('Gbytes : ', size_in_Gbytes)



Graph memory size indices, targets
byte:  2870400
Kbytes :  2803.125
Mbytes :  2.7374267578125
Gbytes :  0.0026732683181762695


# Make a network that represents the Graph
Here each node in the graph is a neuron and each edge is a synaptic connection

In [4]:
from brian2_loihi import *

# Change to numpy to skip compilation time.
# For path backtracing only numpy or cython can be used as codegen.target
# as only when used those you can refer to the synapses and objects a state monitor gives back to you.
# CPP standalone will loose the object references of Python
# which is obvious since the simulation happens in C++ and only the values are returned.
from brian2 import *
prefs.codegen.target = 'numpy' # 'cython'

In [5]:
# ------------------------------
#       Define network 
# ------------------------------
# how many nodes/neurons are present in the lattice Graph?
num_nodes = N*N

# define nodes as neurons
neuron_nodes = LoihiNeuronGroup(
    num_nodes,
    refractory=63,      # set the refractory relatively high to prevent spiking
    threshold_v_mant=1, # 64 = 1*2**6
    decay_v=3000,       # set the voltage decay relatively high to not sum up 2 much
    decay_I=4095        # set the input to zero after every time step
)

# create source and target list from adjacency matrix
sources, targets = fully_connected_nxn_graph_it(N)

# define anti hebbian learning rule
# this will leave a trace from the wavefront algorithm
# inside the connection strength of the synaptic connections.
anti_hebb_learn = 'x0*y1 - y0*x1'

# define synapses with learning rule and proper synaptic traces
syn_connection = LoihiSynapses(source=neuron_nodes,
                               target=neuron_nodes,
                               dw = anti_hebb_learn,
                               w_exp = 0,
                               imp_x1 = 127,
                               tau_x1 = 10,
                               imp_y1 = 127,
                               tau_y1 = 10,
                               sign_mode = synapse_sign_mode.MIXED) # could be also 2 = excitatory
syn_connection.connect(i=sources, j=targets)
syn_connection.w = 2 # which is 128 = 2*2**6


# let the target spike at time step t = 0
# create a spike generator for that
generator_target_init = LoihiSpikeGeneratorGroup(1,
                         indices=[0],
                         times=[0])

# connect this spike generator properly
syn_target_init = LoihiSynapses(generator_target_init,
                                neuron_nodes,
                                sign_mode=synapse_sign_mode.EXCITATORY)
syn_target_init.connect(i=0, j=target_neuron)
syn_target_init.w = 2


# monitor every neuron inside the network with a spike monitor
monitor_all = LoihiSpikeMonitor(neuron_nodes, record=True)

# define a network object and add all previosly defined objects to it
net = LoihiNetwork(neuron_nodes,
                   syn_connection,
                   generator_target_init,
                   syn_target_init,
                   monitor_all)

# Run the network until the wave hits the target

well we know it is a fully connected NxN lattice Graph => we need to run max N*2 Steps! \
Since if we walk on every side that is the maximal path. \
For shorter paths one would not need such long runtime but finding right pathlength can only be done via the workaround described below.\
Unfortunately this slows things down and as such it is just faster to run for the max path length.

In [6]:
#-------------------------------------------
# Workaround for non N*2 runtime (is super slow!):
#-------------------------------------------
# Since the network has to run until the source spikes it is not trivial how long the simualtion runtim shall be.
# Therefore the net runs for 1 step and then checks if the source spiked or not
# this massively slows down the network simulation speed.
# It is therefore a workaround to run the network one time step,
# then check if the target spiked and if not continue running until the target spiked.


# start path search and measure how many steps the net needs to be runned
#net_runtime = 0
#while source_neuron not in monitor_all.i:
#    net.run(1)
#    net_runtime += 1

# make one final step so that weights are updated
#net.run(1)


print('Running wavefront for ',net_runtime, ' steps')



# -------------------------------------------
#    Letting the net run the full runtime
# -------------------------------------------
# run and measure time
start_time = time.time()
net.run(net_runtime, report='text')
SSSP_brian_network_runtime = time.time() - start_time
print('Net needed ', SSSP_brian_network_runtime, ' seconds to simulate')


# -------------------------------------------
#       trace back the path
# -------------------------------------------
# get path length
path_length = max(monitor_all.t)+1 # counting starts at zero therefore add 1
SSP_path_length_brian = path_length

# array for the path
SSP_path_brian = np.zeros(path_length+1, dtype=int)

# save weights
weights = syn_connection.w_clipped

# starting point
post_neuron = source_neuron

# measure time
start_time = time.time()

# walk the path until destination reached
i = 0
SSP_path_brian[i] = post_neuron
while post_neuron != target_neuron:
    
    # connections for the post_neuron in form of the weight
    connections = weights[np.where(syn_connection.j == post_neuron)[0]]
    
    # find the connections where the wave propagation changed the weight
    wave_walk = np.where(connections < 2)[0]
    
    # we only want to walk one path if two available choose the first one
    wave_walk = wave_walk[0]
    
    # find the presynaptic neuron of that connection
    # get all indexes with that postsyn neuron and choose the index with the right (wave_walk) connection
    conn_index = np.where(syn_connection.j == post_neuron)[0][wave_walk]
    
    # now we know the presyn neuron as it is at the same index in the syn.i array
    pre_neuron = syn_connection.i[conn_index]
    
    # we walked the path, start from the new stand point
    post_neuron = pre_neuron
    
    # update path
    i += 1
    SSP_path_brian[i] = post_neuron

SSSP_brian_pathtracingtime = time.time() - start_time
print('path backtracing took ', SSSP_brian_pathtracingtime, ' seconds')

Running wavefront for  600  steps
Starting simulation at t=0. s for a duration of 0.6 s
259. ms (43%) simulated in 10s, estimated 13s remaining.
0.519 s (86%) simulated in 20s, estimated 3s remaining.
0.6 s (100%) simulated in 23s
Net needed  23.56303644180298  seconds to simulate
path backtracing took  5.206444263458252  seconds


# Run the network via C++ standalone code generation
This letting the exact same network run via C++ standalone device.\
Here C++ code will be generated and run external to Python.\
The results will be given back to brian.\
Unfortunately path back tracing as it is implemented above does not work with this device since all the Pyhtone object references to synapses and neurons will be lost.\
It can be seen as speed test for the simulator to how much simulation speed is gained when using the C++ device.

In [7]:
from brian2_loihi import *

# change to C++ standalone to have a faster runtime
from brian2 import *
set_device('cpp_standalone', directory=brian_code_dir, debug=True, run=False, compile=True, clean=True)

In [8]:
# ------------------------------
#       Define network 
# ------------------------------
# how many nodes are present?
num_nodes = N*N

# define nodes as neurons
neuron_nodes = LoihiNeuronGroup(
    num_nodes,
    refractory=63,
    threshold_v_mant=1, # 64 = 1*2**6
    decay_v=3000,       # like in pathfiinding module
    decay_I=4095        # like in pathfiinding module
)

# create source and target list from adjacency matrix
sources, targets = fully_connected_nxn_graph_it(N)

# define anti hebbian learning rule
# this will leave a trace from the wavefront algorythm inside the connection strength.
anti_hebb_learn = 'x0*y1 - y0*x1'

# define synapses
syn_connection = LoihiSynapses(source=neuron_nodes,
                               target=neuron_nodes,
                               dw = anti_hebb_learn,
                               w_exp = 0,
                               imp_x1 = 127,
                               tau_x1 = 10,
                               imp_y1 = 127,
                               tau_y1 = 10,
                               sign_mode = synapse_sign_mode.MIXED) # could be also 2 = excitatory
syn_connection.connect(i=sources, j=targets)
syn_connection.w = 2


# let the target spike at time step t=0
generator_target_init = LoihiSpikeGeneratorGroup(1,
                         indices=[0],
                         times=[0])
syn_target_init = LoihiSynapses(generator_target_init,
                                neuron_nodes,
                                sign_mode=synapse_sign_mode.EXCITATORY)
syn_target_init.connect(i=0, j=target_neuron)
syn_target_init.w = 2


# monitor everything
monitor_all = LoihiSpikeMonitor(neuron_nodes, record=True)

net = LoihiNetwork(neuron_nodes,
                   syn_connection,
                   generator_target_init,
                   syn_target_init,
                   monitor_all)

# ------------------------------
#     Compile, run and measure time 
# ------------------------------
# define custom cpp report function that writes execution time to a file we can read later on to get stuff here into jupyther python
report_func = '''
    //opening a file in writing mode which is default.
    ofstream file;
    file.open("cpp_output.txt");
    
    int remaining = (int)((1-completed)/completed*elapsed+0.5);
    if (completed == 0.0)
    {
        std::cout << "Starting simulation at t=" << start << " s for duration " << duration << " s"<<std::flush;
    }
    if (completed == 1.0)
    {
        std::cout << "] " <<elapsed <<"s elapsed"<<std::flush;
        
        file<< elapsed << endl;

        //We need to close every file which you open.
        file.close();
    }
    if ((completed != 1.0) && (completed != 0.0))
    {
        int barWidth = 70;
        std::cout << "\\r[";
        int pos = barWidth * completed;
        for (int i = 0; i < barWidth; ++i) {
                if (i < pos) std::cout << "=";
                else if (i == pos) std::cout << ">";
                else std::cout << " ";
        }
        std::cout << "] " << int(completed * 100.0) << "% completed. | "<<elapsed <<"s elapsed"<<std::flush;
        

    }
'''


# ------------------------------
# compile the stuff (we set the cpp_satndalone device to run=False,
# therefore this code here will only compile the C++ code but does not run it automatically!)
# ------------------------------
start_time = time.time()
net.run(net_runtime, report=report_func)
SSSP_brian_network_cpp_compile_time = time.time() - start_time
print('net compiletime was', SSSP_brian_network_cpp_compile_time, ' seconds')


# ------------------------------
# run the compiled C++ programm
# ------------------------------
import subprocess
# open the cpp programm which is compiled as main
p = subprocess.Popen("cd Cpp_target_code && ./main", shell=True, stdout = subprocess.PIPE)
output,err = p.communicate()
# print(output)


# ------------------------------
# get the running time which was written by the cpp programm to a file
# ------------------------------
f = open(brian_code_dir+'/cpp_output.txt', "r")
SSSP_brian_network_cpp_run_time = f.read()
print('net runtime was ', SSSP_brian_network_cpp_run_time, ' seconds')

net compiletime was 6.472942352294922  seconds
net runtime was  10.932
  seconds


# Run the network via C++ and openmp
This will use multiple processing cores to speed up the brian simulation.

In [9]:
from brian2_loihi import *

# change to numpy to skip compilation time (takes longer than sim)
from brian2 import *
set_device('cpp_standalone', directory=brian_code_dir, debug=True, run=False, compile=True, clean=True)
prefs.devices.cpp_standalone.openmp_threads = 4

#device.reinit()
#device.activate()


In [10]:
# ------------------------------
#       Define network 
# ------------------------------
# how many nodes are present?
num_nodes = N*N

# define nodes as neurons
neuron_nodes = LoihiNeuronGroup(
    num_nodes,
    refractory=63,
    threshold_v_mant=1, # 64 = 1*2**6
    decay_v=3000,       # like in pathfiinding module
    decay_I=4095        # like in pathfiinding module
)

# create source and target list from adjacency matrix
sources, targets = fully_connected_nxn_graph_it(N)

# define anti hebbian learning rule
# this will leave a trace from the wavefront algorythm inside the connection strength.
anti_hebb_learn = 'x0*y1 - y0*x1'

# define synapses
syn_connection = LoihiSynapses(source=neuron_nodes,
                               target=neuron_nodes,
                               dw = anti_hebb_learn,
                               w_exp = 0,
                               imp_x1 = 127,
                               tau_x1 = 10,
                               imp_y1 = 127,
                               tau_y1 = 10,
                               sign_mode = synapse_sign_mode.MIXED) # could be also 2 = excitatory
syn_connection.connect(i=sources, j=targets)
syn_connection.w = 2


# let the target spike at time step t=0
generator_target_init = LoihiSpikeGeneratorGroup(1,
                         indices=[0],
                         times=[0])
syn_target_init = LoihiSynapses(generator_target_init,
                                neuron_nodes,
                                sign_mode=synapse_sign_mode.EXCITATORY)
syn_target_init.connect(i=0, j=target_neuron)
syn_target_init.w = 2


# monitor everything
monitor_all = LoihiSpikeMonitor(neuron_nodes, record=True)

net = LoihiNetwork(neuron_nodes,
                   syn_connection,
                   generator_target_init,
                   syn_target_init,
                   monitor_all)


# ------------------------------
#     Compile, run and measure time 
# ------------------------------
# define custom cpp report function that writes execution time to a file we can read later on to get stuff here into jupyther python
report_func = '''
    //opening a file in writing mode which is default.
    ofstream file;
    file.open("cpp_output.txt");
    
    int remaining = (int)((1-completed)/completed*elapsed+0.5);
    if (completed == 0.0)
    {
        std::cout << "Starting simulation at t=" << start << " s for duration " << duration << " s"<<std::flush;
    }
    if (completed == 1.0)
    {
        std::cout << "] " <<elapsed <<"s elapsed"<<std::flush;
        
        file<< elapsed << endl;

        //We need to close every file which you open.
        file.close();
    }
    if ((completed != 1.0) && (completed != 0.0))
    {
        int barWidth = 70;
        std::cout << "\\r[";
        int pos = barWidth * completed;
        for (int i = 0; i < barWidth; ++i) {
                if (i < pos) std::cout << "=";
                else if (i == pos) std::cout << ">";
                else std::cout << " ";
        }
        std::cout << "] " << int(completed * 100.0) << "% completed. | "<<elapsed <<"s elapsed"<<std::flush;
        

    }
'''

# ------------------------------
# compile the stuff (we set the cpp_satndalone device to run=False,
# therefore this code here will only compile the C++ code but does not run it automatically!)
# ------------------------------
start_time = time.time()
net.run(net_runtime, report=report_func)
SSSP_brian_network_cpp_openmp_compile_time = time.time() - start_time
print('net compiletime was', SSSP_brian_network_cpp_openmp_compile_time, ' seconds')


# ------------------------------
# run the compiled C++ programm
# ------------------------------
import subprocess
p = subprocess.Popen("cd Cpp_target_code && ./main", shell=True, stdout = subprocess.PIPE)
output,err = p.communicate()
print(output)


# ------------------------------
# get the running time which was written by the cpp programm to a file
# ------------------------------
f = open(brian_code_dir+'/cpp_output.txt', "r")
SSSP_brian_network_cpp__openmp_run_time = f.read()
print()
print()
print('cpp openmp runtime was ', SSSP_brian_network_cpp__openmp_run_time, ' seconds') 

WARNING    OpenMP code is not yet well tested, and may be inaccurate. [brian2.devices.cpp_standalone.device.openmp]


net compiletime was 8.66246747970581  seconds
b'Starting simulation at t=0 s for duration 0.6 s\r[===============================================================>      ] 91% completed. | 10.0031s elapsed] 10.9424s elapsedStarting simulation at t=0 s for duration 0.6 s] 3.5325s elapsedNumber of synapses: 358800\nNumber of synapses: 1\nNumber of synapses: 358800\nNumber of spikes: 0\nNumber of synapses: 358800\nNumber of synapses: 1\nNumber of synapses: 358800\nNumber of spikes: 90000\n'


cpp openmp runtime was  3.5325
  seconds


# Print the runtime and results

In [14]:
print('Numpy   run time:    ', SSSP_brian_network_runtime,'s')
print('C++     compile time:', SSSP_brian_network_cpp_compile_time)
print('C++     run time:    ', SSSP_brian_network_cpp_run_time)
print('openmp  compile time:', SSSP_brian_network_cpp_openmp_compile_time)
print('openmp  run time:    ', SSSP_brian_network_cpp__openmp_run_time)
print('Path backtrace time: ', SSSP_brian_pathtracingtime)
print()
print('-------')
print()
print('Shortest path length:', SSP_path_length_brian)
print('Path nodes:', SSP_path_brian[::-1])

Numpy   run time:     23.56303644180298 s
C++     compile time: 6.472942352294922
C++     run time:     10.932

openmp  compile time: 8.66246747970581
openmp  run time:     3.5325

Path backtrace time:  5.206444263458252

-------

Shortest path length: 599
Path nodes: [    0     0     1     2     3     4     5     6     7     8     9    10
    11    12    13    14    15    16    17    18    19    20    21    22
    23    24    25    26    27    28    29    30    31    32    33    34
    35    36    37    38    39    40    41    42    43    44    45    46
    47    48    49    50    51    52    53    54    55    56    57    58
    59    60    61    62    63    64    65    66    67    68    69    70
    71    72    73    74    75    76    77    78    79    80    81    82
    83    84    85    86    87    88    89    90    91    92    93    94
    95    96    97    98    99   100   101   102   103   104   105   106
   107   108   109   110   111   112   113   114   115   116   117   118
 